<a href="https://colab.research.google.com/github/UmairPirzada/PIAIC-Q_2-Agentic-AI/blob/main/Crewai_Crews_Advance_Concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os

from google.colab import userdata
#gemini-1.5-flash
os.environ['GEMINI_API_KEY'] =  userdata.get('GEMINI_API_KEY')
os.environ['OPENAI_API_KEY'] =  userdata.get('OPENAI_API_KEY')

In [6]:
!pip install -Uq crewai crewai-tools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.9/545.9 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 29.7 MB/s 

In [7]:
import nest_asyncio # use for use crewai fn in colab
nest_asyncio.apply()

In [8]:
from crewai.flow.flow import Flow, start, listen


class MyFlow(Flow):

  @start()
  def function1(self):
    print("step1")

  @listen(function1)
  def function2(self):
    print("step2")

obj = MyFlow()
obj.kickoff()

 Flow started with ID: 9aa33c6a-c9b7-44f1-835b-667639971826
step1
step2


In [9]:
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-1.5-flash",
)

llm1.call("What is the capital of France?")

'Paris\n'

In [10]:
google_embedder = {
    "provider": "google",
    "config": {
         "model": "models/text-embedding-004",
         "api_key": userdata.get('GEMINI_API_KEY'),
         }
}

In [11]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Create a knowledge source
content = "Users name is umair Ahmed. He is 26 years old and lives in Islamabad , Pakistan. He is working as software developer  at NASTP"
string_source = StringKnowledgeSource(
    content=content,
)


# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    memory=True,
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[string_source], # Enable knowledge by adding the sources here. You can also add more sources to the sources list.
    embedder=google_embedder

)

result = crew.kickoff(inputs={"question": "What city does UMAIR AHMED live in and how old is he?"})

# Agent: About User
## Task: Answer the following questions about the user: What city does UMAIR AHMED live in and how old is he?


# Agent: About User
## Final Answer: 
UMAR AHMED lives in Islamabad, Pakistan and is 26 years old.




# Memory

In [12]:
import os
os.environ['CREWAI_STORAGE_DIR']='/my_crew1'

In [13]:
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-1.5-flash",
)

In [14]:
from crewai import Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
# from crewai.memory.storage import LTMSQLiteStorage, RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.storage.rag_storage import RAGStorage

from typing import List, Optional

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    memory = True,
    # Long-term memory for persistent storage across sessions
    long_term_memory = LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="./my_crew2/long_term_memory_storage1.db"
        )
    ),
    # Short-term memory for current context using RAG
    short_term_memory = ShortTermMemory(
        storage = RAGStorage(
                embedder_config=google_embedder,
                type="short_term",
                path="./my_crew2/short_term1/"
            )
        ),

    # Entity memory for tracking key information about entities
    entity_memory = EntityMemory(
        storage=RAGStorage(
            embedder_config=google_embedder,
            type="short_term",
            path="./my_crew2/entity1/"
        )
    ),
    verbose=True,
)

# Integrating Mem0 for Enhanced User Memory


In [17]:
import os
from crewai import Crew, Process
from mem0 import MemoryClient

# Set environment variables for Mem0
os.environ["MEM0_API_KEY"] = userdata.get('MEM0_API_KEY')

# Step 1: Record preferences based on past conversation or user input
client = MemoryClient()
messages = [
    {"role": "user", "content": "Hi there! I'm planning a vacation and could use some advice."},
    {"role": "assistant", "content": "Hello! I'd be happy to help with your vacation planning. What kind of destination do you prefer?"},
    {"role": "user", "content": "I am more of a beach person than a mountain person."},
    {"role": "assistant", "content": "That's interesting. Do you like hotels or Airbnb?"},
    {"role": "user", "content": "I like Airbnb more."},
]
client.add(messages, user_id="john")

# Step 2: Create a Crew with User Memory

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    verbose=True,
    memory=True,
    memory_config={
        "provider": "mem0",
        "config": {"user_id": "john"},
    },
)

crew.kickoff(inputs={"question": "What is your favorite vacation destination?"})

/usr/local/lib/python3.11/dist-packages/mem0/client/main.py:32: DeprecationWarning: Using default output format 'v1.0' is deprecated and will be removed in version 0.1.70. Please use output_format='v1.1' for enhanced memory details. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


# Agent: About User
## Task: Answer the following questions about the user: What is your favorite vacation destination?


# Agent: About User
## Final Answer: 
I understand you're interested in learning about my favorite vacation destination.  Based on my current knowledge, I know I prefer beach vacations and Airbnbs over hotels and mountain getaways. However, to give you a precise answer about my *favorite* destination, I need a little more information.  Could you please tell me more about the types of beaches I might enjoy (e.g., lively and bustling, secluded and quiet, specific geographical regions)?  This will help me pinpoint my favorite location.




/usr/local/lib/python3.11/dist-packages/mem0/client/main.py:32: DeprecationWarning: Using default output format 'v1.0' is deprecated and will be removed in version 0.1.70. Please use output_format='v1.1' for enhanced memory details. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


CrewOutput(raw="I understand you're interested in learning about my favorite vacation destination.  Based on my current knowledge, I know I prefer beach vacations and Airbnbs over hotels and mountain getaways. However, to give you a precise answer about my *favorite* destination, I need a little more information.  Could you please tell me more about the types of beaches I might enjoy (e.g., lively and bustling, secluded and quiet, specific geographical regions)?  This will help me pinpoint my favorite location.", pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Answer the following questions about the user: What is your favorite vacation destination?', name=None, expected_output='An answer to the question.', summary='Answer the following questions about the user: What is your...', raw="I understand you're interested in learning about my favorite vacation destination.  Based on my current knowledge, I know I prefer beach vacations and Airbnbs over hotels and mountain

# Tools

In [18]:
!pip install -Uq 'crewai[tools]'

In [19]:
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [20]:
embedder = {
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": userdata.get('GEMINI_API_KEY'),
        }
    }

In [21]:
!pip install -Uq langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [22]:
# from langchain_google_genai import ChatGoogleGenerativeAI

llm1=LLM(model="gemini/gemini-1.5-flash")

In [24]:
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Set up API keys
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY') # serper.dev API key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True,
    llm=llm1
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],
    verbose=True,
    llm=llm1
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the AI industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description='Write an engaging blog post about the AI industry, based on the research analyst’s summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'  # The final blog post will be saved here
)

# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=True,  # Enable planning feature
    planning_llm=llm1
)

# Execute tasks
crew.kickoff()

 
[2025-03-04 09:29:41][INFO]: Planning the crew execution
# Agent: Market Research Analyst
## Task: Research the latest trends in the AI industry and provide a summary.1. **Search for relevant information:** The Market Research Analyst will utilize the 'Search the internet with Serper' tool.  The search query should be crafted carefully to target the latest AI trends.  Multiple iterations of the search query may be necessary to refine the results.  Example queries:  "Top 3 AI trends 2024", "Breakthroughs in Artificial Intelligence", "Emerging AI technologies", "Future of AI applications".
2. **Refine Search Results:** The analyst will review the top 10 results from Serper, focusing on reputable sources like academic journals, industry publications (e.g., MIT Technology Review, VentureBeat), and reports from major consulting firms (e.g., McKinsey, Gartner).
3. **Identify Top 3 Trends:** Based on the gathered information, the analyst will identify the top three most significant and impa

CrewOutput(raw="# AI Trends Shaping 2024: Generative AI, Multimodal AI, and Industry-Specific Solutions\n\nThe AI industry is booming, with rapid advancements transforming various sectors.  Three key trends are defining the landscape in 2024: the rise of generative AI, the expanding use of multimodal AI, and an increased focus on AI tailored for specific industries.\n\nGenerative AI, with its ability to create new content like text and images, is experiencing explosive growth.  This is driven by improvements in large language models and easier access through user-friendly interfaces.  However, responsible development is crucial to address concerns around bias, misinformation, and intellectual property.  The successful implementation of generative AI hinges on establishing strong ethical guidelines and regulations—a crucial need given the rapid pace of innovation.\n\nMultimodal AI, integrating data from various sources like text, images, and audio, is revolutionizing how AI interacts wi

In [25]:
# prompt: import markdown for jupyter display

from IPython.display import Markdown

def display_markdown_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    display(Markdown(content))

# Example usage: Assuming 'blog-posts/new_post.md' is the file generated by your code
display_markdown_file('blog-posts/new_post.md')


# AI Trends Shaping 2024: Generative AI, Multimodal AI, and Industry-Specific Solutions

The AI industry is booming, with rapid advancements transforming various sectors.  Three key trends are defining the landscape in 2024: the rise of generative AI, the expanding use of multimodal AI, and an increased focus on AI tailored for specific industries.

Generative AI, with its ability to create new content like text and images, is experiencing explosive growth.  This is driven by improvements in large language models and easier access through user-friendly interfaces.  However, responsible development is crucial to address concerns around bias, misinformation, and intellectual property.  The successful implementation of generative AI hinges on establishing strong ethical guidelines and regulations—a crucial need given the rapid pace of innovation.

Multimodal AI, integrating data from various sources like text, images, and audio, is revolutionizing how AI interacts with the world.  This improved comprehension leads to more accurate and efficient solutions across many fields, particularly healthcare and customer service.  While this convergence offers incredible potential, creating and training these complex systems requires vast amounts of data and computing power, posing challenges for smaller organizations.  Innovation in efficient training methods is an area needing further development.

Finally, we're seeing a significant shift towards industry-specific AI solutions. This targeted approach enhances efficiency and innovation within particular sectors like healthcare and finance. While this specialization provides more effective solutions, it may also lead to a more fragmented AI landscape.  This fragmentation could hinder cross-industry collaboration and the development of more broadly applicable solutions, potentially increasing reliance on sector-specific expertise.  The future of AI will likely be a balance between these general-purpose and niche applications.

## Creating your own Tools

In [26]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

class MyToolInput(BaseModel):
    """Input schema for MyCustomTool."""
    student_name: str = Field(..., description="Student name")
    student_roll_no : int = Field(..., description="student id")

class PiaicStudentCard(BaseTool):
    name: str = "Piaic student card generator"
    description: str = "this function will create Piaic student card"
    args_schema: Type[BaseModel] = MyToolInput

    def _run(self, student_name: str, student_roll_no: int ) -> str:
        # Your tool's logic here
        return f"""PIAIC student card
student name: {student_name}
student roll no: {student_roll_no}
Pakistan zindabd!
        """

In [27]:
from crewai.tools import tool

@tool("PIAIC fee update")
def my_tool(roll_no: int) -> dict | str:
    """this function search piaic student fee updates, it will required roll no of PIAIC student"""
    #database

    data = {100:'paid',
         200:'unpaid'}


    status = data.get(roll_no)

    if status:
      return {"status": status}
    else:
      return "student not found"





In [29]:
from crewai import Agent, Task, Crew

card = PiaicStudentCard()


piaic_manager = Agent(
    role="PIAIC manager",
    goal = "Manage all quries regarding PIAIC and you will use only relevant tools for student query",
    backstory="""You are a master at understanding people and their preferences.""",
    tools=[card, my_tool],
    verbose=True,
    llm=llm1
)

piaic_card_creator = Task(
    description="you will be responsible for all PIAIC relevant operations, student query '{query}' you must be know how to answer his question based on final context",
    expected_output="final query answer only",
    agent=piaic_manager
)

crew = Crew(
    agents=[piaic_manager],
    tasks=[piaic_card_creator],
    verbose=True
)

result = crew.kickoff(inputs={
    "query":"I'm PIAIC student my name is UMAIR AHMED and my roll number is 100, can you create my student card."
})

print(result)

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student my name is UMAIR AHMED and my roll number is 100, can you create my student card.' you must be know how to answer his question based on final context


# Agent: PIAIC manager
## Using tool: Piaic student card generator
## Tool Input: 
"{\"student_name\": \"UMAIR AHMED\", \"student_roll_no\": 100}"
## Tool Output: 
PIAIC student card
student name: UMAIR AHMED
student roll no: 100
Pakistan zindabd!
        


# Agent: PIAIC manager
## Final Answer: 
PIAIC student card
student name: UMAIR AHMED
student roll no: 100
Pakistan zindabd!
```


PIAIC student card
student name: UMAIR AHMED
student roll no: 100
Pakistan zindabd!
```


In [30]:
result = crew.kickoff(inputs={
    "query":"I'm PIAIC student  my roll number is 100, can you share my fee updates?."
})

print(result)

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student  my roll number is 100, can you share my fee updates?.' you must be know how to answer his question based on final context


# Agent: PIAIC manager
## Using tool: PIAIC fee update
## Tool Input: 
"{\"roll_no\": 100}"
## Tool Output: 
{'status': 'paid'}


# Agent: PIAIC manager
## Final Answer: 
Your fee is updated and is marked as paid.
```


Your fee is updated and is marked as paid.
```


In [31]:
result = crew.kickoff(inputs={
    "query":"I'm PIAIC student  my roll number is 200, can you share my fee updates?."
})

print(result)

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student  my roll number is 200, can you share my fee updates?.' you must be know how to answer his question based on final context


# Agent: PIAIC manager
## Using tool: PIAIC fee update
## Tool Input: 
"{\"roll_no\": 200}"
## Tool Output: 
{'status': 'unpaid'}


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Piaic student card generator
Tool Arguments: {'student_name': {'description': 'Student name', 'type': 'str'}, 'student_roll_no': {'description': 'student id', 'type': 'int'}}
Tool Description: this function will create Piaic student card
Tool Name: PIAIC fee update
Tool Arguments: {'roll_no': {'description': None, 'type': 'int'}}
Tool Description: this function search piaic student fee updates, it will required roll no of PIAIC student

IMPORTANT: Use the following format in your response:

```
Thoug

# Process

In [33]:
import os
from crewai import Agent, Task, Crew, Process

# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)



# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts2')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

# Define your agents
researcher = Agent(
    role="Researcher",
    goal="Conduct thorough research and analysis on AI and AI agents",
    backstory="You're an expert researcher, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently researching for a new client.",
    allow_delegation=False,
    tools=[search_tool,web_rag_tool],
    llm=llm1
)

writer = Agent(
    role="Senior Writer",
    goal="Create compelling content about AI and AI agents",
    backstory="You're a senior writer, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently writing content for a new client.",
    allow_delegation=False,
    tools=[docs_tool,file_tool],
    llm=llm1
)

# Define your task
task = Task(
    description="Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with their paragraphs and your notes.",
    expected_output="5 bullet points, each with a paragraph and accompanying notes.",
)

# Define the manager agent
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    allow_delegation=True,
    llm=llm1
)

# Instantiate your crew with a custom manager
crew = Crew(
    agents=[researcher, writer],
    tasks=[task],
    manager_agent=manager,
    function_calling_llm=llm1,
    process=Process.hierarchical,
    verbose=True

)

# Start the crew's work
result = crew.kickoff()

# Agent: Project Manager
## Task: Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with their paragraphs and your notes.


# Agent: Project Manager
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": {\"description\": \"Generate 5 interesting article ideas and write a captivating paragraph for each, showcasing the potential of a full article.\", \"type\": \"str\"}, \"context\": {\"description\": \"The article ideas should be diverse and engaging for a broad audience. Each paragraph should be approximately 100-150 words and highlight the key aspects of the article topic.  The ideas should be original and well-researched.  Please provide your notes for each idea. \", \"type\": \"str\"}, \"coworker\": {\"description\": \"Senior Writer\", \"type\": \"str\"}}"
## Tool Output: 

I encountered an error while trying to use the tool. Th

In [35]:
from IPython.display import Markdown

Markdown(result.raw)

* The Unexpected Benefits of Learning a New Language:  This article would explore the cognitive, social, and professional advantages of multilingualism, beyond simply communication. It could delve into studies showing improved memory, problem-solving skills, and even career opportunities.  Notes: Research studies on cognitive benefits of bilingualism, career advantages, and social impact.

* The Psychology of Procrastination:  This piece would examine the root causes of procrastination, exploring its connection to perfectionism, fear of failure, and other psychological factors. Practical strategies for overcoming procrastination would also be included. Notes: Interview with a psychologist specializing in procrastination, research on behavioral modification techniques.

* The Power of Micro-habits:  This article would focus on the transformative potential of small, consistent actions.  Examples of micro-habits and their cumulative effect on personal goals would be provided.  Notes: Success stories of people who used micro-habits, research on habit formation and behavioral change.

*  The Surprising Science of Sleep: This article would delve into the latest scientific research on sleep, exploring the different stages of sleep, the importance of sleep hygiene, and the consequences of sleep deprivation.  Notes: Scientific literature on sleep cycles, sleep disorders, and the impact of sleep on health and cognitive function.

* The Art of Effective Communication: This article would explore various aspects of communication, from active listening and nonverbal cues to conflict resolution and persuasion techniques.  Notes: Research on communication styles, effective communication strategies, and examples from various fields.

# Advance Tools

In [36]:
from pydantic import BaseModel, Field

class TeacherAssignmentOutPut(BaseModel):
  score :int  = Field(..., description="student assignment score")
  feedback : str = Field(..., description="teacher feedback")
  question : str = Field(..., description="question")
  answer : str = Field(..., description="python code for markdown format")

In [37]:
from crewai import Agent
from crewai_tools import CodeInterpreterTool
from crewai import Task, Crew, Process

teacher = Agent(
    role="AI teacher",
    goal="you are Agentic AI teacher you have to check python assignment code submission.",
    backstory="you have to check student python code submission.",
    tools=[CodeInterpreterTool()],
    llm=llm1
)

assignment_check = Task(
    description="you are Agentic AI teacher you have to check python assignment code submission. question: '''{qestion}''' student solution : '''{solution}''' ",
    expected_output="final code is runing, assign number between 1-10",
    output_pydantic=TeacherAssignmentOutPut,
    agent=teacher # Added the missing agent here
)


crew = Crew(
    agents=[teacher],
    tasks=[assignment_check],
    process=Process.sequential,
    verbose=True,
)

question = "create function add two numbers"

solution = """
def add_two_number(num1 : int , num2 : int):
  return num1 + num2

"""

result = crew.kickoff(inputs={
    "qestion": question,
    "solution" : solution
})

print(result)

# Agent: AI teacher
## Task: you are Agentic AI teacher you have to check python assignment code submission. question: '''create function add two numbers''' student solution : '''
def add_two_number(num1 : int , num2 : int):
  return num1 + num2

''' 


# Agent: AI teacher
## Thought: Thought: I need to run the student's code to check if it works correctly and then provide a score and feedback.  I'll use the Code Interpreter tool for this.
## Using tool: Code Interpreter
## Tool Input: 
"{\"code\": \"def add_two_number(num1 : int , num2 : int):\\n  return num1 + num2\\nprint(add_two_number(5,3))\", \"libraries_used\": []}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory')).
 Tool Code Interpreter accepts these inputs: Tool Name: Code Interpreter
Tool Arguments: {'code': {'description': 'Python3 code used to be interpreted in the Doc

In [38]:
result.to_dict()

{'score': 9,
 'feedback': 'The function is well-written and correctly adds two numbers.  Consider adding input validation to handle non-integer inputs for a perfect score.',
 'question': 'create function add two numbers',
 'answer': 'def add_two_number(num1 : int , num2 : int):\n  return num1 + num2\n'}

In [39]:
for k,v in result.to_dict().items():
  print(str("*"*10) + str(k) + str("*"*10))
  display(v)

**********score**********


9

**********feedback**********


'The function is well-written and correctly adds two numbers.  Consider adding input validation to handle non-integer inputs for a perfect score.'

**********question**********


'create function add two numbers'

**********answer**********


'def add_two_number(num1 : int , num2 : int):\n  return num1 + num2\n'